In [ ]:
import numpy as np
import os
import PIL
import tensorflow as tf
import random
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.python.framework.ops import enable_eager_execution
import cv2

#disable_eager_execution()
enable_eager_execution()

In [ ]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

In [ ]:
from __future__ import print_function
import argparse
from tensorflow.keras.layers import Input
import scipy.misc

from configs import bcolors
from utils import *
import tensorflow_model_optimization as tfmot


from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
import tensorflow_datasets as tfds
from tiny_imagenet import TinyImagenetDataset

tiny_imagenet_builder = TinyImagenetDataset()

# this call (download_and_prepare) will trigger the download of the dataset
# and preparation (conversion to tfrecords)
#
# This will be done only once and on next usage tfds will
# use the cached version on your host.
#
# You can pass optional argument to this method called
# DownloadConfig (https://www.tensorflow.org/datasets/api_docs/python/tfds/download/DownloadConfig)
# to customize the location where the dataset is downloaded, extracted and processed.
tiny_imagenet_builder.download_and_prepare()

train_dataset = tiny_imagenet_builder.as_dataset(split="train")
validation_dataset = tiny_imagenet_builder.as_dataset(split="validation")

assert(isinstance(train_dataset, tf.data.Dataset))
assert(isinstance(validation_dataset, tf.data.Dataset))

# print info about the data
#print(tiny_imagenet_builder.info)

In [ ]:
print(tiny_imagenet_builder.info)

In [ ]:
train_images = []
train_labels = []
ds_numpy = tfds.as_numpy(train_dataset)
for item in ds_numpy:
    image, label, id = item["image"], item["label"], item["id"]
    train_images.append(keras.applications.vgg16.preprocess_input(cv2.resize(image, (224,224), interpolation = cv2.INTER_AREA)))
    train_labels.append(label)
    break
    
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
np.max(train_labels)

In [ ]:
test_images = []
test_labels = []
ds_numpy = tfds.as_numpy(validation_dataset)
for item in ds_numpy:
    image, label, id = item["image"], item["label"], item["id"]
    test_images.append(np.expand_dims(keras.applications.vgg16.preprocess_input(image),axis=0))
    test_labels.append(label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
train_labels  = tf.keras.utils.to_categorical(train_labels , num_classes=200)
train_labels  = tf.keras.utils.to_categorical(train_labels , num_classes=200)

# Build Models

In [ ]:
# input image dimensions
img_rows, img_cols = 224 ,224
input_shape = (img_rows, img_cols, 3)

K.clear_session()
# define input tensor as a placeholder
input_tensor_1 = Input(shape=[img_rows, img_cols, 3],dtype=tf.float32 )
input_tensor_2 = Input(shape=[img_rows, img_cols, 3])

# load multiple models sharing same input tensor
K.set_learning_phase(0)
model1 = VGG16(input_tensor=input_tensor_1)
model2 = VGG16(input_tensor=input_tensor_2)

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
model2 = quantize_model(model2)

# Load All Images (1000)

In [ ]:
import re
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f)]

In [ ]:
img_paths = list_pictures('./seeds/', ext='JPEG')
target_images = []
for img_path in img_paths:
    gen_img = preprocess_image(img_path)
    orig_img = gen_img.copy()
    # first check if input already induces differences
    pred1, pred2 = model1.predict(gen_img), model2.predict(gen_img)
    label1, label2 = np.argmax(pred1[0]), np.argmax(pred2[0])
    if label1 == label2:
        target_images.append(orig_img)

In [ ]:
len(target_images)

In [ ]:
with open('./test_images.npy', 'wb') as f:
    np.save(f, target_images[0])
    np.save(f, target_images[1])
    np.save(f, target_images[2])
    np.save(f, target_images[3])

# Load Images with Same Label (4?)

In [ ]:
with open('ImageNetLabels.txt') as f:
    classes = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
classes = [x.strip() for x in classes] 

In [ ]:
target_images = []
with open('test_images.npy', 'rb') as f:
    target_images.append(np.load(f, encoding='bytes',allow_pickle=True))
    target_images.append(np.load(f, encoding='bytes',allow_pickle=True))
    target_images.append(np.load(f, encoding='bytes',allow_pickle=True))
    target_images.append(np.load(f, encoding='bytes',allow_pickle=True))

In [ ]:
len(target_images)

# Display Images

In [ ]:
gen_img = target_images[1]
pil_img = tf.keras.preprocessing.image.array_to_img(gen_img[0,:,:,:])

In [ ]:
from IPython.display import Image
display(pil_img)

In [ ]:
pred1, pred2 = model1.predict(gen_img), model2.predict(gen_img)
label1, label2 = np.argmax(pred1[0]), np.argmax(pred2[0])

In [ ]:
tf.keras.applications.resnet.decode_predictions(pred1, top=5)[0]

In [ ]:
tf.keras.applications.resnet.decode_predictions(pred2, top=5)[0]

In [ ]:
model1.summary()

In [ ]:
model2.summary()

# Generate 

In [ ]:
weight_diff = 1
orig_label = label1

In [ ]:
quantized_input = model2.get_layer('input_2').input
loss2 = K.mean(model2.get_layer('quant_predictions').output[..., orig_label])
grads =tf.gradients(loss2, quantized_input)[0]
iterate = K.function([input_tensor_2], [grads])

In [ ]:
grads

In [ ]:
input_ = model1.get_layer('input_1').input
loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
grads = tf.gradients(loss1, input_tensor_1)[0]
iterate = K.function([input_tensor_1], [grads])

In [ ]:
grads

In [ ]:
grads_value = iterate(gen_img)

In [ ]:
input_ = model1.get_layer('input_1').input
loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
grads = tf.gradients(loss1, input_)[0]
iterate = K.function([input_tensor_1], [grads])

In [ ]:
grads

In [ ]:
grads_value = iterate(gen_img.astype(int))

In [ ]:
loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
loss2 = K.mean(model2.get_layer('quant_predictions').output[..., orig_label])
quantized_input = model2.get_layer('quant_block1_conv1').input

# we compute the gradient of the input picture wrt this loss
grads = normalize(tf.gradients(loss1, input_tensor)[0] - tf.gradients(loss2, quantized_input)[0])

# this function returns the loss and grads given the input picture
iterate = K.function([input_tensor], [grads])

In [ ]:
transformation = 'light'
start_point = (0, 0)
occlusion_size =(50, 50)
step = 10
grad_iterations = 100000

In [ ]:
orig_img = gen_img.copy() 

In [ ]:
model2.get_layer('quantize_layer').output

In [ ]:
model2.get_layer('input_1').output

In [ ]:
model1.get_layer('input_1').output

In [ ]:
# this function returns the loss and grads given the input picture
layer1 = tf.quantization.fake_quant_with_min_max_vars(
    input_tensor, -123.68000030517578,151.06100463867188, num_bits=8
)
lll = K.function([input_tensor], [layer1])

In [ ]:
layer1

In [ ]:
# we run gradient ascent for 20 steps
for iters in range(0,grad_iterations):
    
    loss_value1, grads_value = iterate(gen_img.astype(float))
    if transformation == 'light':
        grads_value = constraint_light(grads_value)  # constraint the gradients value
    elif transformation == 'occl':
        grads_value = constraint_occl(grads_value, start_point,occlusion_size)  # constraint the gradients value
    elif transformation == 'blackout':
        grads_value = constraint_black(grads_value)  # constraint the gradients value

    gen_img += grads_value * step
    pred1, pred2= model1.predict(gen_img), model2.predict(gen_img)
    label1, label2 = np.argmax(pred1[0]), np.argmax(pred2[0])
    if not label1 == label2:
        gen_img_deprocessed = deprocess_image(gen_img)
        orig_img_deprocessed = deprocess_image(orig_img)
        print("yeah")
        break

In [ ]:
orig_img - gen_img